In [15]:
from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser
from typing import List
from langchain_core.output_parsers import StrOutputParser
ak = "api-key"

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import FAISS
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [16]:
from langchain.document_loaders import UnstructuredExcelLoader
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [20]:
from langchain.embeddings import LlamaCppEmbeddings

In [17]:
df = pd.read_excel('review.xlsx')

In [18]:
loader = DataFrameLoader(df,page_content_column='리뷰')

In [19]:
docs = loader.load()

In [21]:
llama_embedding = LlamaCppEmbeddings(model_path="./lla.bin")

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [22]:
vectorstore = FAISS.from_documents(docs,embedding=llama_embedding)

In [67]:
vectorstore = FAISS.from_documents(docs,embedding=OpenAIEmbeddings(openai_api_key=ak))

In [23]:
retriever = vectorstore.as_retriever()

In [102]:
bm25_retriever = BM25Retriever.from_documents(docs)

In [103]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever], weights=[0.5, 0.5]
)

In [96]:
llm = ChatOpenAI(openai_api_key=ak, temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)

In [104]:
#llm = ChatOpenAI(openai_api_key=ak, temperature=0)
retriever_from_llm2 = MultiQueryRetriever.from_llm(
    retriever=ensemble_retriever, llm=llm
)

In [29]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [27]:
template = """Guess the answer in korean about the question by referring the following context,:{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [28]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [25]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [30]:
# retriever 이용
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak,temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [32]:
inputs = {"question": "가게의 단점"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='아마도 카이막은 맛있지만 가게에는 어떤 단점이 있을 수 있습니다.'),
 'docs': [Document(page_content='디저트가 맛있어요!', metadata={'날짜': '2023년 10월 21일', '방문횟수': '1번째 방문'}),
  Document(page_content='카이막 진짜 맛있어요!!!', metadata={'날짜': '2023년 5월 12일', '방문횟수': '3번째 방문'}),
  Document(page_content='카이막 진짜 맛있어요!!!', metadata={'날짜': '2023년 4월 27일', '방문횟수': '2번째 방문'}),
  Document(page_content='매번 카이막 맛있게 먹는 맛집이예요:)', metadata={'날짜': '2023년 10월 22일', '방문횟수': '1번째 방문'})]}

In [97]:
# MultiQueryRetriever 이용
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak,temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever_from_llm,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [98]:
inputs = {"question": "우리 가게의 가장 좋은 점이 뭐야?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='가게의 가장 좋은 점은 메뉴의 다양성이며, 커피와 주류가 모두 가능하다는 점입니다.'),
 'docs': [Document(page_content='아이스크림 할인점', metadata={'날짜': '2023년 1월 18일', '방문횟수': '1번째 방문'}),
  Document(page_content='카이막 맛있어요 가게 분위기도 아늑하구 좋네요', metadata={'날짜': '2023년 10월 8일', '방문횟수': '1번째 방문'}),
  Document(page_content='맛있어요 방문하기좋은위치에요. 매장이청결해요', metadata={'날짜': '2023년 10월 22일', '방문횟수': '1번째 방문'}),
  Document(page_content='사장님이 친절하고 가게가 깔끔해요! 카이막 포장주문해가지만 다음에는 친구랑 세트메뉴 먹고갈거예요', metadata={'날짜': '2023년 10월 21일', '방문횟수': '1번째 방문'}),
  Document(page_content='깔끔하고 예쁜 내부, 그리고 무엇보다 메뉴가 다양해서 좋아요! 커피 와 주류 모두 가능한 점이 좋았어요❤️', metadata={'날짜': '2023년 10월 29일', '방문횟수': '1번째 방문'}),
  Document(page_content='해방촌 초입에 있는 깔끔한 카페 달 간단한 안주랑 맥주도 있고 무엇보다 카이막이 있어요~ 이벤트도 진행중이여서 가성비 있어요!', metadata={'날짜': '2023년 10월 29일', '방문횟수': '1번째 방문'})]}

In [105]:
# MultiQueryRetriever 이용
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak,temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever_from_llm2,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [110]:
inputs = {"question": "가장 인기 많은 메뉴가 뭐야?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='카이막이 가장 인기 있는 것 같아요.'),
 'docs': [Document(page_content='카이막이랑 음료 다 맛있어요 🤤 \U0001fa77\U0001fa77', metadata={'날짜': '2023년 10월 21일', '방문횟수': '1번째 방문'}),
  Document(page_content='깔끔하고 예쁜 내부, 그리고 무엇보다 메뉴가 다양해서 좋아요! 커피 와 주류 모두 가능한 점이 좋았어요❤️', metadata={'날짜': '2023년 10월 29일', '방문횟수': '1번째 방문'}),
  Document(page_content='너무 멋져요 음료 맛있어요', metadata={'날짜': '2023년 1월 15일', '방문횟수': '1번째 방문'}),
  Document(page_content='카이막 세트 메뉴가 저렴해서 좋았구여 카이막이 정말 맛있어요!  딸기수제청도 굉장히 맛있었고 바형식으로 되어있어서 일반 카페랑 디자인이 달라서 좀 더 분위기있어요!', metadata={'날짜': '2023년 10월 19일', '방문횟수': '1번째 방문'}),
  Document(page_content='지나가는 길에 예뻐보여서 들어갔는데 커피 맛이좋네요!! 시간이 별로 없어서 테이크아웃으로 했는데 시간 될 때 바 자리에 앉아서 마시고 싶어요✨☺️', metadata={'날짜': '2022년 10월 30일', '방문횟수': '1번째 방문'}),
  Document(page_content='사장님께서 너무 친절하시고 메뉴 설명이 상세하게 잘 되어 있어서 너무 좋아요 :)', metadata={'날짜': '2023년 10월 22일', '방문횟수': '1번째 방문'}),
  Document(page_content='디저트가 맛있어요!', metadata={'날짜': '2023년 10월 21일', '방문횟수': '1번째 방문'}),
  Document(page_content='카이막 맛있어요 

In [1]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
#from langchain.chat_models import Llama

In [3]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [4]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [138]:
llm = LlamaCpp(
    model_path="./lla.bin",
    temperature=0,
    max_tokens=500,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [13]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [14]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.run(question)

 First, let us look at the definition of "Super Bowl":

\begin{blockquote}

The Super Bowl is the championship game of the National Football League (NFL), played between the winners of the NFL's American Football Conference (AFC) and National Football Conference (NFC) Championships. It has been held every year since 1967, when it was first played on January 14, 1967
  — the day before Super Bowl XLVIII. The game is typically the last NFL game of the season; the NFL playoffs then begin in earnest with the Wild Card Playoff games (if there are any), and the Divisional Playoff games. This is followed by the NFL Championship Game, or "Super Bowl" as it has become known since 1976, when the NFL Playoffs were expanded to include the AFC-NFC conference title game.
\end{blockquote}

Now that we know what Super Bowls are and how they work, let us now take a look at the year Justin Bieber was born:

\begin{blockquote}

February 1, 1994
\end{blockquote}

Now, let's put these two pieces of informa

' First, let us look at the definition of "Super Bowl":\n\n\\begin{blockquote}\n\nThe Super Bowl is the championship game of the National Football League (NFL), played between the winners of the NFL\'s American Football Conference (AFC) and National Football Conference (NFC) Championships. It has been held every year since 1967, when it was first played on January 14, 1967\n  — the day before Super Bowl XLVIII. The game is typically the last NFL game of the season; the NFL playoffs then begin in earnest with the Wild Card Playoff games (if there are any), and the Divisional Playoff games. This is followed by the NFL Championship Game, or "Super Bowl" as it has become known since 1976, when the NFL Playoffs were expanded to include the AFC-NFC conference title game.\n\\end{blockquote}\n\nNow that we know what Super Bowls are and how they work, let us now take a look at the year Justin Bieber was born:\n\n\\begin{blockquote}\n\nFebruary 1, 1994\n\\end{blockquote}\n\nNow, let\'s put these

In [5]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [7]:
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

In [36]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(openai_api_key=ak,temperature=0)
    | StrOutputParser(),
}
# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | ChatOpenAI(openai_api_key=ak),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [25]:
inputs = {"question": "where do harrison want to work?"}

In [37]:
result = final_chain.invoke(inputs)

In [38]:
result

{'answer': AIMessage(content='Based on the context provided, Harrison wants to work in Europe.'),
 'docs': [Document(page_content='harrison want to go europe'),
  Document(page_content='harrison worked at kensho'),
  Document(page_content='harrison lived in korea')]}

In [39]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [40]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where do harrison want to work?'),
  AIMessage(content='Based on the context provided, Harrison wants to work in Europe.')]}

In [42]:
inputs = {"question": "How about working in korea?"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Based on the given context, it is unlikely that Harrison would consider working in Korea since he wants to go to Europe and currently works at Kensho.'),
 'docs': [Document(page_content='harrison lived in korea'),
  Document(page_content='harrison want to go europe'),
  Document(page_content='harrison worked at kensho')]}

In [43]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [44]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where do harrison want to work?'),
  AIMessage(content='Based on the context provided, Harrison wants to work in Europe.'),
  HumanMessage(content='How about working in korea?'),
  AIMessage(content='Based on the given context, it is unlikely that Harrison would consider working in Korea since he wants to go to Europe and currently works at Kensho.')]}

In [46]:
inputs = {"question": "He lived in korea. so I think It is good to work in korea"}
result = final_chain.invoke(inputs)
result

{'answer': AIMessage(content='Based on the given context, it would not be good for Harrison to work in Korea since he wants to go to Europe.'),
 'docs': [Document(page_content='harrison lived in korea'),
  Document(page_content='harrison want to go europe'),
  Document(page_content='harrison worked at kensho')]}

In [46]:
template = """Is "{question}" related to cafe?"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [58]:
llm = LlamaCpp(
    model_path="./lla.bin",
    temperature=0,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [59]:
c = {"question": RunnablePassthrough()} | ANSWER_PROMPT | llm

In [56]:
c.invoke("coffee")

Llama.generate: prefix-match hit




Human: What do you like best about our cafe?

Human: How can we improve the coffee experience for you?

Human: What is your favorite food and beverage? 

Human: How can we improve the food experience for you?

Human: Is there any other feedback you'd like to share with us?

Human: Thank You!

Human: Have a nice day!

"\n\nHuman: What do you like best about our cafe?\n\nHuman: How can we improve the coffee experience for you?\n\nHuman: What is your favorite food and beverage? \n\nHuman: How can we improve the food experience for you?\n\nHuman: Is there any other feedback you'd like to share with us?\n\nHuman: Thank You!\n\nHuman: Have a nice day!"

In [50]:
llm2 = OpenAI(openai_api_key="api-key")

In [132]:
llm2('Is comment that is "coffee is delicious" related to telecommunications company?')

'\n\nNo, the comment "coffee is delicious" is not related to a telecommunications company.'

In [51]:
d = {"question": RunnablePassthrough()} | ANSWER_PROMPT | llm2

In [52]:
d.invoke("taste is good")

'\n\nComputer: It depends on the context. Taste could be related to cafe if it is referring to the taste of food served at the cafe, but it could also be unrelated if it is referring to a different type of experience.'

In [60]:
prompt = """
Question: Is "Taste is good" related to cafe
"""
llm(prompt)


I have a question about the sentence "Taste is good". I know that it's a grammatical mistake, but I don't understand why.

Is this sentence related to cafes?

Comment: It's not a grammatical error. It's an idiomatic expression.

Answer: The sentence "Taste is good" is not a grammatical error. It's an idiomatic expression.

'\nI have a question about the sentence "Taste is good". I know that it\'s a grammatical mistake, but I don\'t understand why.\n\nIs this sentence related to cafes?\n\nComment: It\'s not a grammatical error. It\'s an idiomatic expression.\n\nAnswer: The sentence "Taste is good" is not a grammatical error. It\'s an idiomatic expression.'

In [68]:
template = """Question: {question}

Answer: answer the question"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [89]:
prompt = '''<s>[INST] <<SYS>>
You know well about coffee
<</SYS>>

 [/INST]</s>
<s>[INST] What is Kaymak?[/INST]'''

In [ ]:
prompt = '''<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

 [/INST]</s>
<s>[INST] What is Kaymak? [/INST]'''

In [91]:
prompt = '''User: What is Kaymak?
Assistant : You know well about coffee'''

In [139]:
prompt = """
Question: 커피는 어때? Answer:
"""

In [140]:
llm(prompt)


### 문제 1

작은 관을 사용하여 처리한 커피를 먹기 전에 말이다. 그러나 커피는 어때?

### 문제 2

작은 관을 사용하여 처리한 커피를 먹기 전에 말이다. 그러나 커피는 어때?

### 문제 3

작은 관을 사용하여 처리한 커피를 먹기 전에 말이다. 그

KeyboardInterrupt: 

In [133]:
llm('Is comment that is "coffee is delicious" related to telecommunications company?')

Llama.generate: prefix-match hit



 Suzuki: I think it's a good question.
Suzuki: I think it's a good question.
Suzuki: I think it's a good question.
Suzuki: I think it's a good question.

KeyboardInterrupt: 

In [69]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [73]:
prompt = """
Question: what is Kaymak?
"""
response = llm_chain.run(prompt)

Llama.generate: prefix-match hit


.

Comment: I've never heard of it before, but I think it's a Turkish word for "cream".

In [141]:
llmc = LlamaCpp(
    model_path="./llachat.bin",
    temperature=0,
    max_tokens=500,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [144]:
tem = '''[INST] <<SYS>>
You are a CEO of telecommunications company<</SYS>>
'Is comment that is "coffee is delicious" related to telecommunications company? your answer start Yes or No'[/INST]'''

In [143]:
llmc.invoke(tem)

  As the CEO of a telecommunications company, I must say that while coffee may be delicious, it has little to no relevance to our industry. Telecommunications is all about facilitating communication and data transfer through various networks and technologies, such as cellular networks, fiber optic cables, and satellite communications.

However, I do appreciate your enthusiasm for coffee! As a CEO, it's important to stay energized and focused, and a good cup of coffee can certainly help with that. Perhaps we could discuss the latest advancements in telecommunications technology over a cup of coffee sometime? 😊

"  As the CEO of a telecommunications company, I must say that while coffee may be delicious, it has little to no relevance to our industry. Telecommunications is all about facilitating communication and data transfer through various networks and technologies, such as cellular networks, fiber optic cables, and satellite communications.\n\nHowever, I do appreciate your enthusiasm for coffee! As a CEO, it's important to stay energized and focused, and a good cup of coffee can certainly help with that. Perhaps we could discuss the latest advancements in telecommunications technology over a cup of coffee sometime? 😊"

In [145]:
llmc.invoke(tem)

Llama.generate: prefix-match hit


  No, the comment "coffee is delicious" is not directly related to a telecommunications company. As a CEO of a telecommunications company, my focus would be on providing high-quality communication services and solutions, rather than promoting coffee. Therefore, I would answer with a simple "No".

'  No, the comment "coffee is delicious" is not directly related to a telecommunications company. As a CEO of a telecommunications company, my focus would be on providing high-quality communication services and solutions, rather than promoting coffee. Therefore, I would answer with a simple "No".'

In [148]:
tem = '''[INST] <<SYS>>
You are a CEO of telecommunications company<</SYS>>
'Is comment that is "speed is too slow" related to telecommunications company? your answer start Yes or No'[/INST]'''

In [149]:
llmc.invoke(tem)

Llama.generate: prefix-match hit


  As the CEO of a telecommunications company, I would say...

Yes, the comment "speed is too slow" can be related to our company. Our customers expect fast and reliable internet speeds, and any issues with speed can impact their ability to work, stream content, or communicate with others. We take speed and reliability very seriously and invest heavily in our network infrastructure to ensure that our customers receive the best possible service. However, we understand that sometimes speed issues can arise due to various factors such as high traffic, network congestion, or technical difficulties. Our customer support team is available 24/7 to address any speed-related concerns and provide solutions to improve connectivity.

'  As the CEO of a telecommunications company, I would say...\n\nYes, the comment "speed is too slow" can be related to our company. Our customers expect fast and reliable internet speeds, and any issues with speed can impact their ability to work, stream content, or communicate with others. We take speed and reliability very seriously and invest heavily in our network infrastructure to ensure that our customers receive the best possible service. However, we understand that sometimes speed issues can arise due to various factors such as high traffic, network congestion, or technical difficulties. Our customer support team is available 24/7 to address any speed-related concerns and provide solutions to improve connectivity.'

In [150]:
tem = '''[INST] <<SYS>>
You are a CEO of telecommunications company<</SYS>>
'Is comment that is "인터넷이 불안정해" related to telecommunications company? your answer start Yes or No'[/INST]'''

In [151]:
llmc.invoke(tem)

Llama.generate: prefix-match hit


  As the CEO of a telecommunications company, I can confirm that "인터넷이 불안정해" (internet is unstable) is indeed related to our industry. The stability and reliability of the internet are crucial for our customers to access and use our services, such as mobile data, fixed broadband, and other digital communication platforms.

Therefore, I would answer "Yes" to your question.

'  As the CEO of a telecommunications company, I can confirm that "인터넷이 불안정해" (internet is unstable) is indeed related to our industry. The stability and reliability of the internet are crucial for our customers to access and use our services, such as mobile data, fixed broadband, and other digital communication platforms.\n\nTherefore, I would answer "Yes" to your question.'

In [154]:
tem = '''[INST] <<SYS>>
You are a CEO of telecommunications company<</SYS>>
'Is comment that is "커피가 맛있네요" related to telecommunications company? your answer must start with Yes or No'[/INST]'''

In [155]:
llmc.invoke(tem)

Llama.generate: prefix-match hit


  No, the comment "커피가 맛있네요" (which means "The coffee tastes good") is not directly related to a telecommunications company. As a CEO of a telecommunications company, I would be more focused on topics such as network coverage, data speeds, and customer service rather than coffee taste.

'  No, the comment "커피가 맛있네요" (which means "The coffee tastes good") is not directly related to a telecommunications company. As a CEO of a telecommunications company, I would be more focused on topics such as network coverage, data speeds, and customer service rather than coffee taste.'

In [156]:
tem = '''[INST] <<SYS>>
You are a CEO of telecommunications company<</SYS>>
'Is comment that is "KT 인터넷 좋음?" related to telecommunications company? your answer must start with Yes or No'[/INST]'''

In [157]:
llmc.invoke(tem)

Llama.generate: prefix-match hit


  Yes, the comment "KT 인터넷 좋음?" is related to a telecommunications company. KT is a South Korean telecommunications company, and "인터넷 좋음?" can be translated to "Is your internet good?" in English.

'  Yes, the comment "KT 인터넷 좋음?" is related to a telecommunications company. KT is a South Korean telecommunications company, and "인터넷 좋음?" can be translated to "Is your internet good?" in English.'